In [1]:
# Core
import numpy as np
import pandas as pd

# DB
import sqlalchemy
from sqlalchemy import create_engine

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from ztf_data_load import db_engine, load_ztf_classes, load_ztf_objects, load_ztf_detections, ztf_detection_add_dir
# from db_config import db_engine
from db_utils import sp2df

In [3]:
df_cls = load_ztf_classes()

In [4]:
df_cls

,ObjectClassID,ObjectClassName
0,0,Other
1,1,Ceph
2,2,DSCT
3,3,EB
4,4,LPV
5,5,RRL
6,6,SNe
7,7,AGN-I
8,8,Blazar
9,9,CV/Nova


In [9]:
df_obj = load_ztf_objects(0, 1000, 'ZTF17')


In [10]:
df_obj

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,AsteroidProb
0,ZTF17aaaaaal,106,58367.465868,59275.263218,59.899022,50.278084,16.440174,15.665651,16.427925,15.675536,0.991538
1,ZTF17aaaaaam,69,58455.358530,59278.157500,60.275190,49.811785,18.715262,16.550849,18.763388,16.572272,0.937527
2,ZTF17aaaaaan,222,58343.446620,59292.203206,64.334170,45.594256,16.544040,16.052844,16.547027,16.052882,0.869584
3,ZTF17aaaaaao,138,58340.493391,59292.203206,63.989763,45.430490,17.023956,16.095724,17.035832,16.088380,0.799161
4,ZTF17aaaaaas,200,58348.490567,59292.158542,68.578095,49.087084,17.309472,16.562732,17.320121,16.566059,0.562905
...,...,...,...,...,...,...,...,...,...,...,...
995,ZTF17aaaafxh,500,58344.475255,59292.166238,82.629663,32.175141,17.291135,16.236467,17.304867,16.233893,0.919372
996,ZTF17aaaafxk,175,58347.448368,59292.206030,82.492257,32.131850,17.856744,16.927300,17.866957,16.936085,0.865683
997,ZTF17aaaafxt,2,58660.448403,58829.312546,33.720315,30.026622,16.310497,14.205728,16.478888,14.231841,0.575160
998,ZTF17aaaafxv,3,58721.456852,59127.405174,27.846646,30.382350,15.761253,14.454487,15.703148,14.513006,0.955649


In [15]:
df_obj.iloc[105:110]

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,AsteroidProb
105,ZTF17aaaaalg,14,58315.404676,59210.110914,322.216570,45.588921,16.580629,15.774332,16.604777,15.784315,0.978871
106,ZTF17aaaaalh,135,58288.455069,59219.115637,322.684860,45.301219,17.010044,16.349877,17.037022,16.358285,0.771089
107,ZTF17aaaaali,28,58293.476667,59202.160023,322.781628,45.454687,16.840763,15.912118,16.846028,15.919175,0.612402
108,ZTF17aaaaall,92,58290.424907,59223.139549,320.596098,48.335998,16.874866,15.621473,16.873303,15.648763,0.930158
109,ZTF17aaaaalm,17,58372.310706,59028.468403,320.470442,48.492629,NaN,16.133761,NaN,16.144079,0.965826


In [17]:
df_obj.iloc[105:110].fillna(-1.0)

,ObjectCD,ObservationCount,mjd0,mjd1,MeanRA,MeanDEC,MeanMag_g,MeanMag_r,MeanMagPSF_g,MeanMagPSF_r,AsteroidProb
105,ZTF17aaaaalg,14,58315.404676,59210.110914,322.216570,45.588921,16.580629,15.774332,16.604777,15.784315,0.978871
106,ZTF17aaaaalh,135,58288.455069,59219.115637,322.684860,45.301219,17.010044,16.349877,17.037022,16.358285,0.771089
107,ZTF17aaaaali,28,58293.476667,59202.160023,322.781628,45.454687,16.840763,15.912118,16.846028,15.919175,0.612402
108,ZTF17aaaaall,92,58290.424907,59223.139549,320.596098,48.335998,16.874866,15.621473,16.873303,15.648763,0.930158
109,ZTF17aaaaalm,17,58372.310706,59028.468403,320.470442,48.492629,-1.000000,16.133761,-1.000000,16.144079,0.965826


In [13]:
np.sum(np.isnan(df_obj.MeanMag_g))

148

In [7]:
df_det = load_ztf_detections(0, 10, 0)

In [8]:
df_det

,DetectionID,ObjectCD,mjd,RA,DEC,MagPSF,MagApp,MagNR,Sigma_RA,Sigma_DEC
0,1457228490615015059,ZTF18adbdodf,59211.228495,76.092334,-13.177226,15.729690,15.630600,13.363000,0.09,0.09
1,1457228490015015080,ZTF19aaeaenk,59211.228495,79.314825,-12.558813,14.679854,14.875300,12.433000,0.09,0.09
2,1442294391315015008,ZTF19aaeaenk,59196.294398,79.314520,-12.558744,15.751100,15.905200,12.277000,0.09,0.09
3,1457228490215015082,ZTF19abxlvtk,59211.228495,78.132314,-13.282117,15.175592,15.220200,12.632999,0.09,0.09
4,1457228490615015065,ZTF20acubhxk,59211.228495,76.438589,-13.304719,15.885690,16.129999,13.796000,0.09,0.09
5,1457228491815015029,ZTF20adahblx,59211.228495,78.402191,-11.033087,15.333986,15.201600,12.617000,0.09,0.09
6,1457228490715015057,ZTF18achtbsa,59211.228495,77.478173,-12.856938,17.131235,17.323700,13.962000,0.09,0.09
7,1457228493515015103,ZTF19aadkdtl,59211.228495,79.321827,-9.557205,15.094650,15.059900,12.924000,0.09,0.09
8,1444339494615015004,ZTF19aadkdtl,59198.339491,79.321482,-9.557297,17.247900,17.927099,12.927000,0.09,0.09
9,1457228493515015098,ZTF19aadkdtn,59211.228495,78.996498,-9.658824,15.467650,15.290600,13.175000,0.09,0.09


In [9]:
df_det.dtypes

DetectionID      int64
ObjectCD        object
mjd            float64
RA             float64
DEC            float64
MagPSF         float64
MagApp         float64
MagNR          float64
Sigma_RA       float64
Sigma_DEC      float64
dtype: object